In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q transformers biopython kaggle
print("✅ Đã cài đặt xong thư viện!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.0 MB/s eta 0:00:00
✅ Đã cài đặt xong thư viện!


In [4]:
import os
from google.colab import files

# Upload file kaggle.json
print("Vui lòng upload file kaggle.json của bạn:")
files.upload()

# Cấu hình Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Vui lòng upload file kaggle.json của bạn:


Saving kaggle.json to kaggle.json


In [5]:
# Tải dữ liệu cuộc thi (Sẽ mất khoảng 1-2 phút)
print("⏳ Đang tải dữ liệu CAFA 6...")
!kaggle competitions download -c cafa-6-protein-function-prediction
!unzip -q cafa-6-protein-function-prediction.zip -d /content/cafa6_data
print("✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data")

⏳ Đang tải dữ liệu CAFA 6...
  0% 0.00/91.2M [00:00<?, ?B/s]
100% 91.2M/91.2M [00:00<00:00, 1.48GB/s]
✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data


In [2]:
import numpy as np
import pandas as pd
import os

# ==============================================================================
# CẤU HÌNH ĐƯỜNG DẪN
# ==============================================================================
FILES = {
    'idx': '/content/drive/MyDrive/cafa/features/taxonomy/train_species_idx.npy',
    'onehot': '/content/drive/MyDrive/cafa/features/taxonomy/train_species_onehot.npy',
    'tsv': '/content/drive/MyDrive/cafa/features/taxonomy/train_species_proteins.tsv'
}

def inspect_npy(name, path):
    print(f"\n📂 ĐANG PHÂN TÍCH: {name} (.npy)")
    print("-" * 50)
    try:
        # Dùng mmap_mode='r' để không load hết vào RAM
        data = np.load(path, mmap_mode='r')

        print(f"   Shape (Kích thước): {data.shape}")
        print(f"   Dtype (Kiểu dữ liệu): {data.dtype}")

        # Lấy mẫu 5 dòng đầu
        if len(data.shape) == 1:
            print(f"   Sample (5 phần tử đầu): {data[:5]}")
            print(f"   Min: {np.min(data)}, Max: {np.max(data)}")
        else:
            print(f"   Sample (Dòng đầu tiên): \n{data[0]}")
            # Kiểm tra xem có phải one-hot thật không (chỉ chứa 0 và 1)
            unique_vals = np.unique(data[0])
            print(f"   Giá trị trong vector: {unique_vals}")

    except Exception as e:
        print(f"❌ Lỗi: {e}")

def inspect_tsv(name, path):
    print(f"\n📂 ĐANG PHÂN TÍCH: {name} (.tsv)")
    print("-" * 50)
    try:
        # Đọc file TSV
        df = pd.read_csv(path, sep='\t')

        print(f"   Shape: {df.shape}")
        print(f"   Columns: {list(df.columns)}")
        print(f"   Dtypes:\n{df.dtypes}")

        print("\n   📋 5 dòng đầu tiên:")
        print(df.head())

        # Thống kê nhanh
        if df.shape[0] > 0:
            print("\n   📊 Thống kê Unique:")
            for col in df.columns:
                print(f"     - {col}: {df[col].nunique()} giá trị duy nhất")

    except Exception as e:
        print(f"❌ Lỗi: {e}")

# ==============================================================================
# CHẠY
# ==============================================================================

# 1. Kiểm tra file Index
inspect_npy('Species Index', FILES['idx'])

# 2. Kiểm tra file One-hot
inspect_npy('Species One-Hot', FILES['onehot'])

# 3. Kiểm tra file Info (TSV)
inspect_tsv('Species Info', FILES['tsv'])


📂 ĐANG PHÂN TÍCH: Species Index (.npy)
--------------------------------------------------
   Shape (Kích thước): (82403,)
   Dtype (Kiểu dữ liệu): int64
   Sample (5 phần tử đầu): [0 0 0 0 0]
   Min: 0, Max: 4242

📂 ĐANG PHÂN TÍCH: Species One-Hot (.npy)
--------------------------------------------------
   Shape (Kích thước): (82403, 8453)
   Dtype (Kiểu dữ liệu): float32
   Sample (Dòng đầu tiên): 
[1. 0. 0. ... 0. 0. 0.]
   Giá trị trong vector: [0. 1.]

📂 ĐANG PHÂN TÍCH: Species Info (.tsv)
--------------------------------------------------
   Shape: (82403, 3)
   Columns: ['protein', 'taxon', 'taxon_idx']
   Dtypes:
protein      object
taxon         int64
taxon_idx     int64
dtype: object

   📋 5 dòng đầu tiên:
  protein  taxon  taxon_idx
0  A0JNW5   9606          0
1  A0JP26   9606          0
2  A0PK11   9606          0
3  A1A4S6   9606          0
4  A1A519   9606          0

   📊 Thống kê Unique:
     - protein: 82403 giá trị duy nhất
     - taxon: 1381 giá trị duy nhất
     - 

In [7]:
import pandas as pd
import os

# ==============================================================================
# CẤU HÌNH ĐƯỜNG DẪN
# ==============================================================================
TSV_PATH = '/content/drive/MyDrive/cafa/features/taxonomy/train_species_proteins.tsv'
FASTA_PATH = '/content/cafa6_data/Train/train_sequences.fasta'

# ==============================================================================
# HÀM CLEAN ID
# ==============================================================================
def get_clean_id(fasta_header):
    """
    Input: >sp|P28818|RGRF1_RAT (hoặc sp|P28818|RGRF1_RAT)
    Output: P28818
    """
    # Xóa dấu > ở đầu nếu có
    header = fasta_header.lstrip('>')

    # Tách bằng dấu gạch đứng |
    parts = header.split('|')

    # Nếu đúng format UniProt (sp|ID|Name) -> Lấy phần giữa (index 1)
    if len(parts) >= 2:
        return parts[1]

    # Nếu không có dấu |, trả về chuỗi gốc (chỉ lấy phần đầu trước khoảng trắng)
    return header.split()[0]

def load_and_clean_fasta_ids(path):
    ids = []
    print(f"⏳ Đang đọc và clean ID từ FASTA: {os.path.basename(path)}...")
    try:
        with open(path, 'r') as f:
            for line in f:
                if line.startswith('>'):
                    raw_header = line.strip()
                    clean_id = get_clean_id(raw_header)
                    ids.append(clean_id)
        return ids
    except Exception as e:
        print(f"❌ Lỗi đọc FASTA: {e}")
        return []

# ==============================================================================
# THỰC HIỆN SO SÁNH (Code cũ nhưng dùng ID đã clean)
# ==============================================================================

# 1. Load ID từ TSV (Taxonomy)
print(f"⏳ Đang đọc TSV: {os.path.basename(TSV_PATH)}...")
df_tsv = pd.read_csv(TSV_PATH, sep='\t')
tsv_ids = df_tsv['protein'].tolist()

# 2. Load ID từ FASTA (Đã Clean)
fasta_ids = load_and_clean_fasta_ids(FASTA_PATH)

# --- DEBUG: In thử vài ID để chắc chắn đã clean đúng ---
print(f"\n🔍 Kiểm tra mẫu ID sau khi clean:")
print(f"   Fasta Raw (Ví dụ): sp|P28818|RGRF1_RAT")
print(f"   Fasta Clean:       {fasta_ids[0] if len(fasta_ids) > 0 else 'Empty'}")
print(f"   TSV Sample:        {tsv_ids[0] if len(tsv_ids) > 0 else 'Empty'}")

# 3. So sánh Số lượng
print(f"\n📊 KẾT QUẢ SO SÁNH:")
print(f"   - Taxonomy (TSV): {len(tsv_ids):,}")
print(f"   - Sequence (FASTA): {len(fasta_ids):,}")

# 4. So sánh Thứ tự (Order Check)
if len(tsv_ids) == len(fasta_ids):
    mismatch_count = 0
    for i, (id1, id2) in enumerate(zip(tsv_ids, fasta_ids)):
        if id1 != id2:
            mismatch_count += 1
            if mismatch_count <= 3:
                print(f"   ⚠️ Lệch tại dòng {i}: TSV='{id1}' != FASTA='{id2}'")

    if mismatch_count == 0:
        print("\n✅ TUYỆT VỜI! Thứ tự và ID khớp nhau 100%.")
        print("👉 Bạn có thể dùng file train_species_onehot.npy ngay lập tức!")
    else:
        print(f"\n❌ CẢNH BÁO: Có {mismatch_count:,} dòng bị lệch thứ tự/ID.")
else:
    # Nếu số lượng khác nhau, tìm tập giao
    common = set(tsv_ids) & set(fasta_ids)
    print(f"\n⚠️ Số lượng không khớp. Số ID chung: {len(common):,}")

⏳ Đang đọc TSV: train_species_proteins.tsv...
⏳ Đang đọc và clean ID từ FASTA: train_sequences.fasta...

🔍 Kiểm tra mẫu ID sau khi clean:
   Fasta Raw (Ví dụ): sp|P28818|RGRF1_RAT
   Fasta Clean:       A0A0C5B5G6
   TSV Sample:        A0JNW5

📊 KẾT QUẢ SO SÁNH:
   - Taxonomy (TSV): 82,403
   - Sequence (FASTA): 82,404

⚠️ Số lượng không khớp. Số ID chung: 82,403


In [8]:
import pandas as pd
import os

# ==============================================================================
# CẤU HÌNH
# ==============================================================================
TSV_PATH = '/content/drive/MyDrive/cafa/features/taxonomy/train_species_proteins.tsv'
FASTA_PATH = '/content/cafa6_data/Train/train_sequences.fasta'

# ==============================================================================
# HÀM CLEAN ID (Sử dụng lại logic cũ)
# ==============================================================================
def get_clean_id(header):
    header = header.strip().lstrip('>')
    parts = header.split('|')
    if len(parts) >= 2:
        return parts[1] # Lấy ID ở giữa (sp|ID|Name)
    return header.split()[0] # Lấy ID đầu tiên

# ==============================================================================
# TÌM ID KHÁC BIỆT
# ==============================================================================
print("⏳ Đang tải danh sách ID...")

# 1. Lấy set ID từ TSV
df = pd.read_csv(TSV_PATH, sep='\t')
tsv_ids_set = set(df['protein'].unique())

# 2. Lấy set ID từ FASTA
fasta_ids_set = set()
with open(FASTA_PATH, 'r') as f:
    for line in f:
        if line.startswith('>'):
            clean_id = get_clean_id(line)
            fasta_ids_set.add(clean_id)

# 3. Tính hiệu (Set Difference)
# Tìm những thằng có trong Fasta mà KHÔNG có trong TSV
missing_in_tsv = fasta_ids_set - tsv_ids_set

# Tìm những thằng có trong TSV mà KHÔNG có trong Fasta (Kiểm tra ngược lại cho chắc)
missing_in_fasta = tsv_ids_set - fasta_ids_set

print("\n" + "="*50)
print("🕵️ KẾT QUẢ TRUY TÌM:")
print("="*50)

if len(missing_in_tsv) > 0:
    print(f"❌ Có {len(missing_in_tsv)} ID nằm trong FASTA nhưng thiếu Taxonomy (TSV):")
    for pid in missing_in_tsv:
        print(f"   👉 {pid}")
else:
    print("✅ Không có ID nào trong FASTA bị thiếu Taxonomy.")

print("-" * 50)

if len(missing_in_fasta) > 0:
    print(f"❌ Có {len(missing_in_fasta)} ID nằm trong TSV nhưng thiếu Sequence (FASTA):")
    for pid in missing_in_fasta:
        print(f"   👉 {pid}")
else:
    print("✅ Không có ID nào trong TSV bị thiếu Sequence.")

⏳ Đang tải danh sách ID...

🕵️ KẾT QUẢ TRUY TÌM:
❌ Có 1 ID nằm trong FASTA nhưng thiếu Taxonomy (TSV):
   👉 A0A0C5B5G6
--------------------------------------------------
✅ Không có ID nào trong TSV bị thiếu Sequence.
